In [16]:
import pandas as pd
import numpy as np 

df = pd.read_csv("../../data/raw_data_binance_BTC_USDT_1h.csv")

In [17]:
df.drop(columns=["Unnamed: 0"], inplace=True)

In [18]:
df

,time_open,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,2017-12-19 15:00:00,18569.52,18650.02,18545.11,18620.01,338.319586,6.292816e+06,2568.0,128.750204,2.395303e+06
1,2017-12-19 16:00:00,18620.02,18695.20,18165.99,18234.89,505.051461,9.298289e+06,4418.0,198.079068,3.645787e+06
2,2017-12-19 17:00:00,18259.40,18259.40,16300.00,16852.00,1831.436840,3.155096e+07,15372.0,827.913384,1.426553e+07
3,2017-12-19 18:00:00,16890.00,17800.00,16550.22,17250.00,1226.949207,2.113936e+07,11082.0,612.086815,1.057065e+07
4,2017-12-19 19:00:00,17214.20,17900.00,17100.00,17550.50,823.862800,1.443043e+07,6158.0,396.400387,6.944349e+06
...,...,...,...,...,...,...,...,...,...,...
39376,2022-06-22 06:00:00,20897.00,20943.17,20551.00,20723.52,2613.774410,5.412753e+07,42193.0,1245.791750,2.580072e+07
39377,2022-06-22 07:00:00,20723.51,20736.68,20421.38,20587.01,4195.854000,8.638902e+07,58827.0,1911.811000,3.937325e+07
39378,2022-06-22 08:00:00,20587.01,20719.21,20388.01,20411.24,3271.915890,6.732365e+07,53095.0,1526.045120,3.140632e+07
39379,2022-06-22 09:00:00,20411.23,20526.67,20330.00,20466.47,3588.324010,7.332763e+07,53154.0,1650.636910,3.372889e+07


In [19]:
from talipp.ohlcv import OHLCVFactory
from talipp.indicators import AccuDist, ADX, ALMA, AO, Aroon, ATR, BB, BOP, CCI, ChaikinOsc, ChandeKrollStop, CHOP, CoppockCurve, DEMA, DonchianChannels, DPO, EMA, EMV, ForceIndex, HMA, \
    Ichimoku, KAMA, KeltnerChannels, KST, KVO, MACD, MassIndex, MeanDev, OBV, PivotsHL, ROC, RSI, ParabolicSAR, SFX, SMA, SMMA, SOBV, \
    StdDev, Stoch, StochRSI, SuperTrend, TEMA, TRIX, TSI, UO, VTX, VWAP, VWMA, WMA

price_types = ["open", "close", "high", "low"]
periods = [4,8,12,16,20,24,36,48,72,96]

close = df["open"].to_numpy()
high =  df["high"].to_numpy()
low = df["low"].to_numpy()
volume = df["volume"].to_numpy()
open = df["open"].to_numpy()

ohlcv = OHLCVFactory.from_matrix2([
    df["open"].to_numpy(),
    df["high"].to_numpy(),
    df["low"].to_numpy(),
    df["close"].to_numpy(),
    df["volume"].to_numpy()]
)

dct = {
    'close': close,
    'high': high,
    'low': low,
    'open': open,
    'volume': volume
}

def shift(df, col_name):
    df[col_name] = df[col_name].shift(df[col_name].isna().sum())

In [22]:
df["CoppockCurve"] = pd.Series(CoppockCurve(11, 14, 10, close))
df["CoppockCurve"] = df["CoppockCurve"].shift(df["CoppockCurve"].isna().sum())

df["KAMA"] = pd.Series(KAMA(14, 2, 30, close))
df["KAMA"] = df["KAMA"].shift(df["KAMA"].isna().sum())

kst_series = pd.Series(KST(10, 10, 15, 10, 20, 10, 30, 15, 9, close))
df["KST_kst"] = pd.Series(np.vectorize(lambda x: x.kst)(kst_series))
df["KST_signal"] = pd.Series(np.vectorize(lambda x: x.signal)(kst_series))
df["KST_kst"] = df["KST_kst"].shift(df["KST_kst"].isna().sum())
df["KST_signal"] = df["KST_signal"].shift(df["KST_signal"].isna().sum())

macd_series = pd.Series(MACD(12, 26, 9, close))
df["MACD_macd"] = pd.Series(np.vectorize(lambda x: x.macd)(macd_series))
df["MACD_signal"] = pd.Series(np.vectorize(lambda x: x.signal)(macd_series))
df["MACD_macd"] = df["MACD_macd"].shift(df["MACD_macd"].isna().sum())
df["MACD_signal"] = df["MACD_signal"].shift(df["MACD_signal"].isna().sum())

df["MassIndex"] = pd.Series(MassIndex(9, 9, 10, ohlcv))
df["MassIndex"] = df["MassIndex"].shift(df["MassIndex"].isna().sum())

df["OBV"] = pd.Series(OBV(ohlcv))
df["OBV"] = df["OBV"].shift(df["OBV"].isna().sum())

df["KVO"] = pd.Series(KVO(34, 55, ohlcv))
df["KVO"] = df["KVO"].shift(df["KVO"].isna().sum())

# df["Pivots"] = pd.Series(PivotsHL(15, 15, ohlcv))
# df["Pivots"] = df["Pivots"].shift(df["Pivots"].isna().sum())

df["VWAP"] = pd.Series(VWAP(ohlcv))
df["VWAP"] = df["VWAP"].shift(df["VWAP"].isna().sum())

df["AccuDist"] = pd.Series(AccuDist(ohlcv))
df["AccuDist"] = df["AccuDist"].shift(df["AccuDist"].isna().sum())

df["BOP"] = pd.Series(BOP(ohlcv))
df["BOP"] = df["BOP"].shift(df["BOP"].isna().sum())

df["VWAP"] = pd.Series(VWAP(ohlcv))
df["VWAP"] = df["VWAP"].shift(df["VWAP"].isna().sum())

df["UO"] = pd.Series(UO(7, 14, 28, ohlcv))
df["UO"] = df["UO"].shift(df["UO"].isna().sum())

df["TSI"] = pd.Series(TSI(13, 25, close))
df["TSI"] = df["TSI"].shift(df["TSI"].isna().sum())

df["AO"] = pd.Series(AO(5, 34, ohlcv))
df["AO"] = df["AO"].shift(df["AO"].isna().sum())

for period in periods:
    df["CCI"+str(period)] = pd.Series(CCI(period, ohlcv))
    shift(df, "CCI"+str(period))    

    df["CHOP"+str(period)] = pd.Series(CHOP(period, ohlcv))
    shift(df, "CHOP"+str(period))    

    df["VTX"+str(period)] = pd.Series(VTX(period, ohlcv))
    shift(df, "VTX"+str(period))  

    df["SOBV"+str(period)] = pd.Series(SOBV(period, ohlcv))
    shift(df, "SOBV"+str(period))  
    
    df["ATR"+str(period)] = pd.Series(ATR(period, ohlcv))
    shift(df, "ATR"+str(period))  

    df["ForceIndex"+str(period)] = pd.Series(ForceIndex(period, ohlcv))
    shift(df, "ForceIndex"+str(period)) 
    
    adx_series = pd.Series(ADX(period, period, ohlcv))
    df["ADX_plus_di"] = pd.Series(np.vectorize(lambda x: x.plus_di)(adx_series))
    df["ADX_minus_di"] = pd.Series(np.vectorize(lambda x: x.minus_di)(adx_series))
    df["ADX_adx"] = pd.Series(np.vectorize(lambda x: x.adx)(adx_series))
    df["ADX_plus_di"] = df["ADX_plus_di"].shift(df["ADX_plus_di"].isna().sum())
    df["ADX_minus_di"] = df["ADX_minus_di"].shift(df["ADX_minus_di"].isna().sum())
    df["ADX_adx"] = df["ADX_adx"].shift(df["ADX_adx"].isna().sum())

    
    df["ForceIndex"+str(period)] = pd.Series(ForceIndex(period, ohlcv))
    shift(df, "ForceIndex"+str(period)) 
    
    for price_type in price_types:
        df[price_type+"_SMA"+str(period)] = pd.Series(SMA(period, dct[price_type]))
        shift(df, price_type+"_SMA"+str(period))

        df[price_type+"_SMMA"+str(period)] = pd.Series(SMMA(period, dct[price_type]))
        shift(df, price_type+"_SMMA"+str(period))

        df[price_type+"_WMA"+str(period)] = pd.Series(WMA(period, dct[price_type]))
        shift(df, price_type+"_WMA"+str(period))

        df[price_type+"_ALMA"+str(period)] = pd.Series(ALMA(period, 0.85, 6, dct[price_type]))
        shift(df, price_type+"_ALMA"+str(period))   

        df[price_type+"_TEMA"+str(period)] = pd.Series(TEMA(period, dct[price_type]))
        shift(df, price_type+"_TEMA"+str(period))   

        df[price_type+"_EMA"+str(period)] = pd.Series(EMA(period, dct[price_type]))
        shift(df, price_type+"_EMA"+str(period)) 

        df[price_type+"_DEMA"+str(period)] = pd.Series(DEMA(period, dct[price_type]))
        shift(df, price_type+"_DEMA"+str(period))  

        df[price_type+"_DPO"+str(period)] = pd.Series(DPO(period, dct[price_type]))
        shift(df, price_type+"_DPO"+str(period)) 

        df[price_type+"_HMA"+str(period)] = pd.Series(HMA(period, dct[price_type]))
        shift(df, price_type+"_HMA"+str(period))

        df[price_type+"_MeanDev"+str(period)] = pd.Series(MeanDev(period, dct[price_type]))
        shift(df, price_type+"_MeanDev"+str(period))  
        
        df[price_type+"_StdDev"+str(period)] = pd.Series(StdDev(period, dct[price_type]))
        shift(df, price_type+"_StdDev"+str(period))  

        df[price_type+"_RSI"+str(period)] = pd.Series(RSI(period, dct[price_type]))
        shift(df, price_type+"_RSI"+str(period))

        df[price_type+"_ROC"+str(period)] = pd.Series(ROC(period, dct[price_type]))
        shift(df, price_type+"_ROC"+str(period))
        
        df[price_type+"_TRIX"+str(period)] = pd.Series(TRIX(period, dct[price_type]))
        shift(df, price_type+"_TRIX"+str(period))

/tmp/ipykernel_13810/230875067.py:120: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_ROC"+str(period)] = pd.Series(ROC(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_TRIX"+str(period)] = pd.Series(TRIX(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:84: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colu

/tmp/ipykernel_13810/230875067.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_DEMA"+str(period)] = pd.Series(DEMA(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_DPO"+str(period)] = pd.Series(DPO(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:108: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

/tmp/ipykernel_13810/230875067.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_TRIX"+str(period)] = pd.Series(TRIX(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["CCI"+str(period)] = pd.Series(CCI(period, ohlcv))
/tmp/ipykernel_13810/230875067.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

/tmp/ipykernel_13810/230875067.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_WMA"+str(period)] = pd.Series(WMA(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:93: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_ALMA"+str(period)] = pd.Series(ALMA(period, 0.85, 6, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

/tmp/ipykernel_13810/230875067.py:111: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_MeanDev"+str(period)] = pd.Series(MeanDev(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_StdDev"+str(period)] = pd.Series(StdDev(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

...

/tmp/ipykernel_13810/230875067.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_TRIX"+str(period)] = pd.Series(TRIX(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:84: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_SMA"+str(period)] = pd.Series(SMA(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:87: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

/tmp/ipykernel_13810/230875067.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_DEMA"+str(period)] = pd.Series(DEMA(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_DPO"+str(period)] = pd.Series(DPO(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:108: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all col

/tmp/ipykernel_13810/230875067.py:123: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_TRIX"+str(period)] = pd.Series(TRIX(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:53: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["CCI"+str(period)] = pd.Series(CCI(period, ohlcv))
/tmp/ipykernel_13810/230875067.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

/tmp/ipykernel_13810/230875067.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_WMA"+str(period)] = pd.Series(WMA(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:93: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_ALMA"+str(period)] = pd.Series(ALMA(period, 0.85, 6, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:96: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining a

/tmp/ipykernel_13810/230875067.py:111: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_MeanDev"+str(period)] = pd.Series(MeanDev(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[price_type+"_StdDev"+str(period)] = pd.Series(StdDev(period, dct[price_type]))
/tmp/ipykernel_13810/230875067.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joi

In [16]:

# print(f'BB: {BB(20, 2, close)[-1]}')
# print(f'ChaikinOsc: {ChaikinOsc(3, 10, ohlcv)[-1]}')
# print(f'ChandeKrollStop: {ChandeKrollStop(10, 2, 9, ohlcv)[-1]}')
# print(f'DonchianChannels: {DonchianChannels(20, ohlcv)[-1]}')
# print(f'Ichimoku: {Ichimoku(26, 9, 52, 52, 26, ohlcv)[-1]}')
# print(f'KeltnerChannels: {KeltnerChannels(20, 26, 1, 1, ohlcv)[-1]}')
# print(f"SAR: {ParabolicSAR(0.02, 0.02, 0.2, ohlcv)[-20:]}")
# print(f'SFX: {SFX(12, 12, 3, ohlcv)[-1]}')
# print(f'Stoch: {Stoch(14, 3, ohlcv)[-1]}')
# print(f'StochRSI: {StochRSI(14, 14, 3, 3, close)[-1]}')
# print(f'SuperTrend: {SuperTrend(10, 3, ohlcv)[-20:]}')


In [24]:
df.dropna(inplace=True)

In [25]:
df.to_csv("../../data/test.csv")

In [26]:
pd.read_csv("../../data/test.csv")


,Unnamed: 0,time_open,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,low_TEMA96,low_EMA96,low_DEMA96,low_DPO96,low_HMA96,low_MeanDev96,low_StdDev96,low_RSI96,low_ROC96,low_TRIX96
0,286,2017-12-31 13:00:00,12650.00,12828.00,12459.00,12542.42,466.369907,5.880672e+06,5346.0,254.863724,...,12487.146665,13471.949800,12872.004505,848.546771,12378.920009,801.943021,985.055177,45.706928,-23.686051,-3.168424
1,287,2017-12-31 14:00:00,12544.99,12831.31,12493.66,12797.46,377.641478,4.789354e+06,4652.0,235.156423,...,12460.293179,13451.778877,12844.448559,1062.644063,12345.706855,794.004648,969.127257,45.793301,-21.240245,-3.288602
2,288,2017-12-31 15:00:00,12797.44,12955.00,12512.00,12552.01,355.128100,4.522712e+06,4523.0,181.167520,...,12436.059947,13432.401992,12818.616588,638.803750,12314.473468,784.739818,948.421027,45.839373,-21.723142,-3.408400
3,289,2017-12-31 16:00:00,12552.01,12700.00,12425.98,12519.64,345.358570,4.332844e+06,4427.0,150.026889,...,12408.019352,13411.651024,12790.197875,590.718125,12283.841746,778.412799,933.968404,45.655455,-20.853631,-3.528202
4,290,2017-12-31 17:00:00,12519.64,12888.88,12483.65,12888.88,328.946751,4.176715e+06,4895.0,163.014102,...,12385.035350,13392.516982,12765.137775,531.740417,12254.117577,775.029601,926.705050,45.802775,-18.143167,-3.647484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39090,39376,2022-06-22 06:00:00,20897.00,20943.17,20551.00,20723.52,2613.774410,5.412753e+07,42193.0,1245.791750,...,20983.802783,20404.584823,20046.289273,520.710625,21224.484049,819.064271,959.872881,47.657769,0.601031,-12.351819
39091,39377,2022-06-22 07:00:00,20723.51,20736.68,20421.38,20587.01,4195.854000,8.638902e+07,58827.0,1911.811000,...,20980.112965,20404.931115,20054.362254,620.383125,21225.850024,818.895521,959.738083,47.283657,-0.095104,-12.181089
39092,39378,2022-06-22 08:00:00,20587.01,20719.21,20388.01,20411.24,3271.915890,6.732365e+07,53095.0,1526.045120,...,20974.307499,20404.582226,20060.899894,263.789271,21220.515851,819.007066,959.820288,47.187291,0.063067,-12.012195
39093,39379,2022-06-22 09:00:00,20411.23,20526.67,20330.00,20466.47,3588.324010,7.332763e+07,53154.0,1650.636910,...,20964.981845,20403.044449,20064.942279,-155.273229,21209.138359,819.075816,959.866167,47.018954,0.038972,-11.845363


In [23]:
df.dropna()

,time_open,open,high,low,close,volume,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,...,low_TEMA96,low_EMA96,low_DEMA96,low_DPO96,low_HMA96,low_MeanDev96,low_StdDev96,low_RSI96,low_ROC96,low_TRIX96
286,2017-12-31 13:00:00,12650.00,12828.00,12459.00,12542.42,466.369907,5.880672e+06,5346.0,254.863724,3.214652e+06,...,12487.146665,13471.949800,12872.004505,848.546771,12378.920009,801.943021,985.055177,45.706928,-23.686051,-3.168424
287,2017-12-31 14:00:00,12544.99,12831.31,12493.66,12797.46,377.641478,4.789354e+06,4652.0,235.156423,2.983848e+06,...,12460.293179,13451.778877,12844.448559,1062.644063,12345.706855,794.004648,969.127257,45.793301,-21.240245,-3.288602
288,2017-12-31 15:00:00,12797.44,12955.00,12512.00,12552.01,355.128100,4.522712e+06,4523.0,181.167520,2.309072e+06,...,12436.059947,13432.401992,12818.616588,638.803750,12314.473468,784.739818,948.421027,45.839373,-21.723142,-3.408400
289,2017-12-31 16:00:00,12552.01,12700.00,12425.98,12519.64,345.358570,4.332844e+06,4427.0,150.026889,1.881453e+06,...,12408.019352,13411.651024,12790.197875,590.718125,12283.841746,778.412799,933.968404,45.655455,-20.853631,-3.528202
290,2017-12-31 17:00:00,12519.64,12888.88,12483.65,12888.88,328.946751,4.176715e+06,4895.0,163.014102,2.070963e+06,...,12385.035350,13392.516982,12765.137775,531.740417,12254.117577,775.029601,926.705050,45.802775,-18.143167,-3.647484
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39376,2022-06-22 06:00:00,20897.00,20943.17,20551.00,20723.52,2613.774410,5.412753e+07,42193.0,1245.791750,2.580072e+07,...,20983.802783,20404.584823,20046.289273,520.710625,21224.484049,819.064271,959.872881,47.657769,0.601031,-12.351819
39377,2022-06-22 07:00:00,20723.51,20736.68,20421.38,20587.01,4195.854000,8.638902e+07,58827.0,1911.811000,3.937325e+07,...,20980.112965,20404.931115,20054.362254,620.383125,21225.850024,818.895521,959.738083,47.283657,-0.095104,-12.181089
39378,2022-06-22 08:00:00,20587.01,20719.21,20388.01,20411.24,3271.915890,6.732365e+07,53095.0,1526.045120,3.140632e+07,...,20974.307499,20404.582226,20060.899894,263.789271,21220.515851,819.007066,959.820288,47.187291,0.063067,-12.012195
39379,2022-06-22 09:00:00,20411.23,20526.67,20330.00,20466.47,3588.324010,7.332763e+07,53154.0,1650.636910,3.372889e+07,...,20964.981845,20403.044449,20064.942279,-155.273229,21209.138359,819.075816,959.866167,47.018954,0.038972,-11.845363
